# CNNでMNISTを行います : [Deep MNIST for Experts](https://www.tensorflow.org/tutorials/mnist/pros/)


In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../datasets/mnist", one_hot=True) 

Extracting ../../datasets/mnist/train-images-idx3-ubyte.gz
Extracting ../../datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ../../datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ../../datasets/mnist/t10k-labels-idx1-ubyte.gz


In [2]:
batch_xs, batch_ys = mnist.train.next_batch(5) # ミニバッチでデータを取得する
print ("data length->", len(batch_xs[0]))

data length-> 784


784なので28*28 pixelsの手書き文字データです

In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# 便利関数の準備
複数の素子と層を作るので、めんどくさくないように便利な関数を準備します

In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## ネットワークを作っていく
inputされるデータを28*28の形に変更しておく

In [6]:
x_image = tf.reshape(x, [-1,28,28,1])

In [7]:
# 最初の層は5x5のpatchにするので、32 featuresを作る
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [8]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [9]:
# 二層目 5x5のpatchで、64 featuresつくる inputは32
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

In [10]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
print(h_pool2) #　これでサイズ見れるの便利

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [11]:
# 最後に全結合のレイヤーを入れる 7*7*64 →1024
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [12]:
# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [13]:
# 出力層
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [14]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

今回はAdamでクロスエントロピーを最小化します

In [15]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [16]:
# 計算が重たいので、epoch数を減らして、batchサイズを増やしました
for i in range(1000):
    batch = mnist.train.next_batch(100)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.09
step 100, training accuracy 0.92
step 200, training accuracy 0.91
step 300, training accuracy 0.94
step 400, training accuracy 0.89
step 500, training accuracy 0.95
step 600, training accuracy 0.91
step 700, training accuracy 0.96
step 800, training accuracy 0.94
step 900, training accuracy 0.95


In [17]:
print("test accuracy %g"%accuracy.eval(feed_dict={ x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9696
